# Coalescent trees exercise

### Learning objectives

By the end of this lesson you will:

1. Be familiar with the Coalescent as a model for generating genealogies.
2. Be introduced to the concept of a probability distribution.
3. Understand the coalescent model as a probability distribution for genealogies.
4. Be able to simulate coalescent genealogies as random variables.
5. Understand the connection between population $N_e$ and coalescent waiting times.

In [ ]:
import toytree
import toyplot
import numpy as np
import pandas as pd
from scipy import stats

### Coalescent trees
A coalescent tree represents the genealogical history of a set of gene copies at a single locus. A coalescent tree exists for any set of homologous gene copies, whether those copies are physically located within a single diploid individual, among many individuals in a single population, or spread among many different populations or species. 
(The case where gene copies are sampled from multiple different populations is the subject of a later exercise, so here we will focus on the case of a coalescent history for a set of sampled gene copies in a single population.) 
A coalescent tree traces a history, backwards in time, from a set of sampled gene copies until each coalesces with another into their common ancestor. Eventually, all gene copies will coalesce into a shared most recent common ancestor (MRCA), from which they are all descended. 

### The Coalescent Model

The coalescent (also called the coalescent model, Kingman coalescent, or n-coalescent) is a model of *neutral* evolution. It is a mathematical approximation to the Wright-Fisher (WF) process, meaning that it is expected to yield genealogies in the same way that stochastic simulations of the WF process generates genealogies. Recall, the WF process implements multiple generations of random mating in a finite size population with non-overlapping discrete generations. These same model assumptions apply to the coalescent model. At the end of a WF simulation you can sample a random number of gene copies at the present and trace back a genealogy connecting them (**Fig. 1A**). This genealogy represents a random outcome of the WF process. If the simulation was run again from a different random seed it would yield a different genealogy for the same set of sampled gene copies. However, if we examine many repetitions of this process, the average times between coalescent events on these genealogies would begin to converge on expectations that can be derived from probability theory.

The coalescent is a shortcut to generating these genealogies. Instead of simulating a whole population forward in time, and then sampling a few samples at the end, we can instead start from the present, by choosing the number of samples we want to study, and simulate *backwards-in-time* a coalescent history for those samples (**Fig. 1B**). To do this, the coalescent uses probability theory to estimate the expected time between coalescent events given information about the population in which evolution is occurring. For a single population, the only required information is the effective population size, $N_e$. <span style="color:tomato; font-weight:600">The power of the coalescent model lies in connecting information contained in coalescent genealogies with information about the population in which the genealogies are embedded.</span>

<div style="border: solid lightgrey 2px; padding: 15px">
    <img src="../docs/images/coalescent-wf.png">
    <p style="text-align: justify">
        Fig.1: A population evolving under a Wright-Fisher process generates a 
        genealogy tracing the history of ancestor-descendant relationships
        among gene copies. (A) A simulated Wright-Fisher process for a population
        of N=15. The genealogy connecting 5 randomly sampled gene copies is shown
        in bold. (B) The same genealogy drawn as a coalescent tree. The expected
        time between coalescent events, which reduce the number of samples left
        to coalesce from k to k-1, can be estimated in each interval using the
        the Kingman coalescent based on N and k.
    </p>
</div>

### The Probability of Coalescence
The coalescent was first developed by Kingman (1982). It is an extension of earlier work by Sewall Wright (1931), who first described the process by which gene copies in a WF process form genealogical patterns. He showed the the probability that two gene copies share a common ancestor one generation in the past is 1/2N. Kingman extended this idea by deriving probability statements for coalescence among any arbitrary number of samples, $k$. By doing so, he laid the groundwork for a probability theory of genealogical trees: the coalescent. The core of this model is a derivation of <span style="color:tomato; font-weight:600">the probability that a coalescent event occurs among $k$ gene copies in a population of size $N_e$ </span> during a single generation of random mating:

$$ P(\text{coal} | N, k) = \frac{k(k-1)}{2} \times \frac{1}{2N} = \frac{k(k-1)}{4N} $$

As you can see, the final solution is derived from a product of two terms. The first, $k(k-1)/2$ is the number of ways to sample two items from a collection of k items, and the second term, $1/2N$, is the probability that two samples coalesce one generation ago. Thus, you can see that this expression is calculating the probability that any pair of gene copies coalesced in one generation.

How does this probability statement help us to develop a probability theory for distributions of genealogies? We will walk through this in several steps, starting with some background on probability theory more generally. To start, let's first conceptualize the question of coalescence as a [Bernoulli random variable](https://en.wikipedia.org/wiki/Bernoulli_distribution) -- a random sample where each trial yields either a success (1) with probability $p$, or a failure (0) with probability 1-$p$. In our context, each trial represents a generation in which a coalescent event either occurs, or doesn't.

We could perform many discrete trials to get a series of results, e.g., `0,0,0,0,1,0,0,0,...`. If the probability of a success is very low then it might take many trials before we observe a success. This is likely to be the case for the coalescent in most practical cases, where $N$ is much larger than $k$. Each generation the probability of a coalescent event among $k$ samples, from a population with a large $N$, is relatively low, and so we may observe hundreds of generations between coalescent events. Let's examine this below, where the probability $p$ is calculated for $N$=1000 and $k$=5. 

In [ ]:
# define the variables N, k, and p
N = 1000
k = 5
p = (k * (k - 1)) / (4 * N)

print(f"P(coal|N,k) = p={p}")

### Random variables
Here we will use a function from the `scipy.stats` Python package to sample Bernoulli random variables. As we can see from the calculation above, the probability of a coalescence one generation ago is only 5/1000, so it is very unlikely we will observe a success (coalescence) from sampling a small number of trials. Run the code cell below several times. Each time shows the results of 100 random trials. (You can increase the size parameter to run more trials at a time.) This should confirm the intuition that the probability of a coalesce event among 5 selected samples from a population of size 1000 during a single generation of random mating is pretty low. But note, it is random. Every trial represents a random variable. The larger the sample size, the closer the result (the frequency of the occurrence of successes) will converge towards the mean expectation, based on the parameter $p$. 

In [ ]:
print(stats.bernoulli.rvs(p=p, size=100))

### Probability distributions

We can now ask a more general question: how many generations does it take until the next coalescent event? This is actually a common type of question in statistics. Because we have a probability statement for a single trial, we can rephrase our question as: what is the expected number of Bernoulli trials before one success is observed? The best way to answer this question is not with a single number, such as the mean, but rather as a [probability distribution](https://en.wikipedia.org/wiki/Probability_distribution) -- in this case, specifically, the geometric probability distribution -- which can tell us the probability of any possible result. <span style="color:tomato; font-weight:600">Probability distributions are functions that yield different results depending on the parameters they are given</span>. Many probability distributions have been defined which serve a major role in probability and statistics. For our purposes, they serve two important roles: for calculating the probability of an observation, and for drawing random variables from a parameterized distribution. As we saw above, the *Bernoulli probability distribution* takes one parameter, $p$. Once we define a distribution, we can sample random variables from it. The probability distribution also has a number of features defined by its parameters, such as its mean, median, and variance. The mean of the Bernoulli distribution is the expected frequency that it will return a success, which is simply $p$. 

### Exponential distribution
Let's now return to our question: <span style="color:tomato; font-weight:600">how many generations does it take until the next coalescent event?</span> The answer to this question will be distributed according to the [Geometric probability distribution](https://en.wikipedia.org/wiki/Geometric_distribution), which is a distribution that describes the number of Bernoulli trials needed to get one success. Technically, we will actually use the [Exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution) for this same purpose in all of our code going forward. This distribution is an analog of the geometric distribution, but returns results in terms of continuous rather than discrete time steps. It's just a bit easier to use. This is demonstrated below, where we sample 10 random variables from each distribution, both parameterized with the same probability of success (note: the exponential function `expon` by convention takes its parameter argument as 1/p).

In [ ]:
# sample 10 random variables from the geometric distribution
stats.geom.rvs(p=p, size=10)

In [ ]:
# sample 10 random variables from the exponential distribution
stats.expon.rvs(scale=1/p, size=10)

Let's examine the  exponential distribution. One easy way to do this is to simply sample many random variables from it and to examine their distribution visually. Below we sample 50K random variables from an exponential distribution with a probability of success equal to $p$. We then plot the results as a histogram. The result shows that shorter waiting times are much more common than longer waiting times under the exponential distribution.

In [ ]:
# randomly sample 50K waiting times from the exponential distribution
random_samples = stats.expon.rvs(scale=1/p, size=50000)

# plot a histogram 
histo = np.histogram(random_samples, bins=50, density=True)
toyplot.bars(
    histo,
    xlabel="Waiting time (n trials)", ylabel="Frequency", 
    label=f"Exponential Distribution w/ p={p}",
    width=400, height=250,
);

### Mean waiting times
The question of "how long until the next event" is common in statistics, and is called a "waiting time". A shorthand description of the exponential distribution is as a "distribution of waiting times". The shape of this distribution will always be exponential, with its rate, or scale, determined by the parameter $p$. It thus has a very simple solution for calculating the mean (also called the Expectation, $E$) which is <span style="color:tomato; font-weight:600"> $ E[x] = 1/p $ </span> (in fact, as mentioned above, this is what the `expon` function takes as a parameter argument). 

What does this mean in terms of waiting times for coalescence? It means that the expected waiting time until the next coalescent which occurs each generation with probability $p$ is also 1/$p$. Taking 1 over some variable is called the reciprocal, because it can be interpreted as flipping the numerator and denominator of a fraction. Thus, we can describe the expected waiting time until the next coalescent event, $E[t_k]$, as the reciprocal of the coalescent probability: 

$$ E[t_k] = \frac{4N}{k(k-1)} $$

### Probability density
Another feature of a probability distribution is its probability density. <span style="color:tomato; font-weight:600">A probability density can be used to calculate the exact probability of a specific observation</span>. For example, we can ask, given a parameterized model (e.g., exponential distribution with p=0.005) what is the probability of this observation (e.g., a waiting time of 500)? The example below calculates the probability of 5 different waiting times. A larger probability indicates that the value is a more likely outcome from this probability density. Notice that the probability of the longer waiting times is lower. This makes sense given the exponential distribution, where the probability of longer waiting times decreases exponentially.

In [ ]:
print(stats.expon.pdf(scale=1/p, x=[10, 50, 100, 200, 500]))

The probability density is especially useful in [maximum likelihood](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) inference, which is a method for estimating the parameters of a model based on observed data. In other words, we might observe 100K waiting times that we assume are random variables sampled from a single exponential distribution. We can then ask which parameters of this distribution are most likely to generate the observed data. The analogous scenario for the coalescent would be to observe thousands of coalescent trees and to ask what $N_e$ value for a population is most likely to produce genealogies with these waiting times. 

Let's first implement a simple visual example of maximum likelihood inference below. We first generate an observed dataset by drawing 100K random variables from an exponential distribution with parameter $p$=0.005, which has a mean waiting time expectation of $E[x]=1/p=200$. We confirm that the mean of the random samples is 200. Then, we calculate the log likelihood of all 100K observations by taking the sum of the log likelihood of each individual waiting time. Finally, this is repeated for many different possible values of $p$. 

We expect that the value for the parameter $p$ that was used to generate the original data (p=0.005) should yield the highest log likelihood. This would mean that the distribution of the observed data is closest to the exponential probability distribution parameterized with that value. In other words, this parameter is most likely to have produced the observed data. The below plot shows the log likelihood for many different values for $p$. It shows a peak at p=0.005, as expected.

In [ ]:
# sample 50K random variables from an exponential distribution with parameter p
observations = stats.expon.rvs(scale=1/p, size=100_000)
print(f"mean of {observations.size} observations is: {observations.mean():.2f}")

In [ ]:
# calculate the likelihood of these observation at many different values for expon(p)
xvalues = np.logspace(-4, -1.75, 200)
loglik = [np.sum(stats.expon.logpdf(scale=1/i, x=observations)) for i in xvalues]
c, a, m = toyplot.plot(
    xvalues, 
    np.array(loglik),
    label="Log Likelihood surface w/ optim at p=0.005",
    ylabel="Log Likelihood", 
    xlabel="Expon. rate parameter (p)",
    width=400, height=250, 
);
a.x.ticks.show = True
a.x.ticks.locator = toyplot.locator.Explicit([0, 0.005, 0.01, 0.015])
a.vlines(0.005, color="tomato");

### Coalescent as a probability distribution
Kingman (1982) showed how we can use the expected waiting times between coalescent events as a probabilistic method to generate random coalescent trees. The important outcome of this realization is that <span style="color:tomato; font-weight:600">we can treat coalescent trees as *random variables.*</span> In other words, we can think of the distribution of possible coalescent trees that could be observed in a population of effective size $N_e$ *as a probability distribution*, which has a predictable mean, median, variance, etc., and from which we can sample genealogies as random variables.

The steps involved with *generating* a coalescent tree as a random variable is a bit more involved than sampling a value from an exponential distribution. But not by much. In fact, it simply involves sampling $k$-1 random variables from $k$-1 exponential distributions with slightly different rate parameters. This is because the rate of coalescence decreases after each subsequent coalescent event; a coalescence decreases the number of gene copies that remain uncoalesced from $k$ to $k$-1. 

Imagine we are constructing a single genealogy as a random variable for the history of $k$ gene copies in a population. **This process should be conceptualized as occurring backwards in time**, starting with 5 samples, then 4, then 3, and finally 2. We don't really care what happens after the final two samples coalesce, because at that point we have reached the MRCA of our $k$ samples, and no further coalescence is possible in our tree.

For visual purposes, below we calculate an array of values for $k$ between 5 and 2, and display a table showing  the corresponding values for $k(k-1)/2$, $P(coal|N,k)$, and $E[t_k]$ for each value of $k$. Note that $N_e$ remains constant. As we can see in row 0, there are initially 5 gene copies, and thus 10 ways in which we could sample two gene copies at a time from 10. This is the number of possible samples that could coalesce in a single generation. Because this number is quite high, the probability of coalescence is also quite high, and so the expected waiting time until a coalescence is low (200). By contrast, look at row 3: here only two gene copies remain, and there is only one way to sample two from two, thus the probability of coalescence is quite low, and correspondingly, the expected waiting time until the next event is very long (2000). 

In [ ]:
# calculate k-choose-2, P(coal), and E[t] for multiple values of k
kvalues = np.arange(k, 1, -1)
kchoose2 = (kvalues * (kvalues - 1)) / 2
prob_coal = (kvalues * (kvalues - 1)) / (4 * N)
waiting_times = 1 / prob_coal

# show a summary as a table
pd.DataFrame({
    "$N_e$": 1000,
    "k": kvalues, 
    "k(k-1)/2": kchoose2, 
    "P(coal | N,k)": prob_coal, 
    "E[$t_k]$": waiting_times,
})

### A function to generate a coalescent tree

Below we define a function, `generate_coal_tree()`, to simulate a coalescent tree as a random variable from a population of effective size $N_e$. This function takes two required arguments: the number of sampled gene copies $k$ and the population size $N_e$. You can optionally provide a seed for the random number generator to make it return the same tree repeatably. It is not important at this point that you understand the Python code itself. Instead, focus on trying to understand the verbal description of the algorithm -- the steps involved. 

This function represents one way in which to implement a coalescent simulation. Coalescent simulators have been written in various programming languages, and using various modifications of the algorithm below, but all follow the same general steps. The steps involved in generating a coalescent tree are as follows:

1. Calculate the $P(coal | N_e, k)$ for the current value of $k$.
2. Sample a waiting time until the next coalescent event as a exponential random variable.
3. Randomly join two gene copies into a common ancestor at the sampled coalescent time, reducing the value of $k$ by 1.
4. Extend the branch lengths of the remaining samples to align with the new node.
5. Repeat from step 1 until all gene copies have coalesced.

In [ ]:
def generate_coal_tree(k: int, Ne: int, seed: int=None) -> toytree.ToyTree:
    """Return a random ToyTree generated under the n-coalescent.
    
    Waiting times between coal events under the n-coalescent are exponentially
    distributed with rate (k * k-1)/4N. A tree is constructed by randomly
    samples waiting times from an exponential for each value of k from k 
    to 1, and randomly joining Nodes at each coalescent interval.
    
    Parameters
    ----------
    k: int
        The number of gene copies sampled at the present.
    Ne: int
        The effective population size (Ne).
    seed: int or None
        A seed for the numpy random number generator.
    """
    # seed rng
    rng = np.random.default_rng(seed)

    # sample exponential waiting times for each value of k
    kvals = np.arange(k, 1, -1)
    lambdas = (4 * Ne) / (kvals * (kvals - 1))
    waiting_times = rng.exponential(lambdas)
    
    # make Nodes for k samples, initially not connected.
    nodes = {i: toytree.Node(name=str(i)) for i in range(k)}
    
    # iterate over coalescent time intervals randomly joining Nodes
    for time in waiting_times:
        # increment the dist for all current Nodes
        for node in nodes:
            nodes[node]._dist += time
            
        # randomly sample two existing Nodes to connect to new parent
        new_node = toytree.Node(name=str(k + 1))
        for _ in range(2):
            node = nodes.pop(rng.choice(tuple(nodes)))
            new_node._add_child(node)

        # advance name counter and add new parent to current tips
        k = k + 1
        nodes[k] = new_node
    return toytree.ToyTree(new_node)

### Coalescent simulations
Run the code block below several times to examine the different genealogies that it produces. Feel free to change the $k$ or $N_e$ value. Examine the type of variation that is produced: how closely related different tip names are, how variable the branch lengths are, how tall the y-axis scale bar is. Each represents a *coalescent simulation* -- a term used to describe sampling a genealogy as a random variable from a probability distribution defined by the coalescent model. 

In [ ]:
# generate a random coalescent tree
tree = generate_coal_tree(k=15, Ne=10000)

# draw the tree
tree.draw(scale_bar=True, layout='d');

### Coalescent variation

Upon examining several simulated trees you will likely find that there doesn't seem to be much in common among them. You can think about this as analagous to sampling 10 different numbers from an exponential distribution. They are expected to exhibit high variance, and by examining a small number of observations it would be very difficult to estimate what the mean of the distribution is. However, upon examining a large number of observations you can begin to estimate the shape of the exponential distribution, and to predict statistics about it, such as its mean. 

The same is true for coalescent trees. You may have noticed that total tree height is somewhat predictable, but the relationships are not. This is because the tree height (sum of all coalescent waiting times) is predicted by parameters of the coalescent model. By contrast, the genealogical relationships among samples is not. <span style="color:tomato; font-weight:600">The relationships among gene copies <span style="text-decoration: underline">in a single population</span> coalescent tree are random because the model assumes a population is randomly mating</span>. (We will see in the next exercise that relationships among gene copies from *different populations* can be non-random.) We thus expect that in a *panmictic* (randomly-mating) population the coalescent trees from different unlinked regions of the genome will trace back completely random, and usually different, genealogical relationships.

By comparing many trees side by side on the same y-axis we can begin to recognize similarities in their relative and total heights (coalescent times). The lengths of waiting times between coalescent events are greater when $N_e$ is greater. And the sum of all waiting times between coalescent events leads to the total tree height. This height is called the time to the most recent common ancestor, or $T_{mrca}$ (**Fig. 1B**). 

<div class="alert alert-block alert-info">
    <h3>Action: Change the value of $N_e$ to examine variation in coalescent trees.</h3>
    <p>
        Execute the code block below several times to examine coalescent variation
        among many trees generated with the same parameter settings (k=10, Ne=10000).
        The trees are all drawn on the same y-axis scale. <b>Try to figure out the
        mean $T_{mrca}$ (time to the common ancestor)</b>. Once you have a
        rough estimate, try to describe this number as a multiple of the $N_e$ 
        parameter.
    </p>
</div>

In [ ]:
# generate 12 random trees
mtree = toytree.mtree([generate_coal_tree(k=10, Ne=10000) for i in range(12)])

# draw all 12 trees
mtree.draw(shape=(3, 4), shared_axes=True, scale_bar=True, layout='d', height=600);

### Time to the most recent common ancestor
Hopefully you reached a solution that the $T_{mrca} \thickapprox$ 4$N_e$. Why do you think this would be? We previously described that the waiting time between coalescent events is explained by the coalescent probability, determined by $N_e$ and $k$. So shouldn't the number of samples in our genealogy matter? If we add more samples shouldn't it take longer for them to coalesce? Somewhat unintuitively, *it turns out that is not the case.* 

The reason for this is that as you add more samples the probability that any two of them coalesce in any generation increases. And so more coalescent events are likely to occur very early on. This leads to the **common shape of coalescent genealogies**, which tend to have many small branches near the present, and fewer long branches. The waiting time between coalescent events is initially very low, but gets longer and longer as fewer samples are left to coalesce. Beyond a certain point, adding more samples just adds more short branches
that are nested within the deeper splits in the tree. We won't delve into the math here, but it is interesting to note that approximately half of the time until the $T_{mrca}$ of a coalescent tree, on average, is spent waiting for just the final two samples to coalesce. 

<div class="alert alert-block alert-info">
    <h3>Action: Change the $N_e$ and/or $k$ to examine variation in the $T_{mrca}$</h3>
    <p>
        The code block below will simulate 500 trees and print the average tmrca. 
        You should find that increasing $k$ to very large values does not make the 
        tmrca much deeper, it remains at approximately 4$N_e$. At small values of $k$
        you will observe a slight underestimation of the $T_{mrca}$.
    </p>
</div>

In [ ]:
# change these values to different integers as you like (but remember to ensure Ne >> k)
K = 50
NE = 100000

# do not change the code below here
many_trees = [generate_coal_tree(k=K, Ne=NE) for i in range(500)]
tmrcas = [i.treenode.height for i in many_trees]
print(f"The estimated T_mrca in a population with Ne={NE} is: {np.mean(tmrcas):.2f}")
print(f"T_mrca is thus approximately a factor of {np.mean(tmrcas)/NE:.2f} x Ne")

### Diminishing returns from sampling

A consequence of the fact that adding more gene copies to a genealogy does not greatly extend its height, is that <span style="color:tomato; font-weight:600">we can usually sample most of the genetic diversity in a population by sampling just a small number of gene copies</span>. This is demonstrated visually below. In this example we simulate a coalescent tree for a large number of samples (e.g., 40) and highlight the genealogy connecting a smaller random subet of those samples (e.g., 10). You will find that the genealogy connecting the 10 samples covers the majority of the total branch lengths of the tree. 
Adding an additional 30 samples does not contribute much new information. There is a diminishing return in the effort to add more samples. 

The function below is used to generate the plots further below. Once again, do not worry about the details of the Python code. 

In [ ]:
def draw_random_tree_with_sampled_path(ntips: int, nsamples: int, Ne: int=100000):
    """Returns drawing of a coalescent tree with path connecting subsample.
    
    Parameters
    ----------
    ntips: int
        The total number of tips in the simulated coalescent tree (k).
    nsamples: int
        The number of subsampled tips to highlight the history of.
    Ne: int
        The effective population size
    """
    assert nsamples <= ntips, "nsamples must be <= ntips"
    tree = generate_coal_tree(k=ntips, Ne=Ne)
    tips = list(np.random.choice(range(ntips), nsamples, replace=False))
    e = tree.get_ancestors(*tips, stop_at_mrca=True, include_top=False)
    n = tree.get_ancestors(*tips, stop_at_mrca=True, include_top=True)
    tree.set_node_data('ecolor', {i: 'red' for i in e}, default="black", inplace=True)
    tree.set_node_data('ncolor', {i: 'red' for i in n}, default="black", inplace=True)
    c, a, m = tree.draw(ts='o', node_mask=False, node_sizes=6, 
                        layout='d', edge_type='p', height=400, scale_bar=True,
                        edge_colors=tree.get_node_data("ecolor"),
                        node_colors=tree.get_node_data("ncolor"));
    return c

<div class="alert alert-block alert-info">
    <h3>Action: Change the total number of samples, and/or subsamples.</h3>
    <p>
        Re-run this simulation multiple times. Notice the tendency for many of the
        coalesce events to occur very quickly, and for the final coalescence events
        to have longer waiting times. The coalescent history connecting a random
        sample of 10 random gene copies is highlighted in red. It is highly likely
        that this genealogy contains most of the same branches that a much larger
        sample would, as demonstrated by the genealogy of the total 40 samples. In 
        the rare cases that you observe long black branches, this indicates genetic
        diversity that exists in the population that we may have missed by sampling
        only 10 gene copies.
        <br><br>
        Remember, sampling 10 gene copies would mean sampling just 5 diploid individuals.
    </p>
</div>

In [ ]:
draw_random_tree_with_sampled_path(ntips=40, nsamples=10, Ne=100_000)

## Summary of coalescent trees

- Wright-Fisher (WF) models describe evolution of a population through random mating, discrete non-overlapping generations, and genetic drift in a finite population size.
- WF models generate genealogies of ancestor-descendant relationships among gene copies.
- The probability that two gene copies coalesce into a common ancestor one generation back is 1/2N.
- Kingman extended this probability statement to describe the probability that one coalescence occurs among $k$ gene copies one generation back: $k(k-1)/4N$.
- A Bernoulli random variable describes the probability of a binary outcome (e.g., coin flip) given a parameter for the probability of a success.
- The exponential probability distribution can describe the waiting time until one success  given a parameter for the probability of a success.
- Probability distributions can be used to generate random variables, and describe the probability of observations. 
- The coalescent can be interpreted as a probability distribution for generating genealogies as random variables, and describing the probability that a population of size $N_e$ produced an observed genealogy.
- Coalescent simulation is very useful because it allows us to describe expected genealogies in a population without needing to simulate the entire population (its super fast). 
- The expected time until all samples in a population coalesce to a common ancestor ($T_{mrca}$) is ~4$N_e$.